In [64]:
import pandas as pd
import os
import json

# Ruta del archivo Excel
filePath = '../sdei-annual-pm2-5/sdei-annual-pm2-5-concentrations-countries-urban-areas-v1-1998-2016-xlsx.xlsx'

# Verificar si el archivo existe
if not os.path.exists(filePath):
    print(f"No se encontró el archivo: {filePath}")
    exit()

# Cargar el archivo Excel
excelData = pd.ExcelFile(filePath)

# Información de las hojas a cargar y los filtros a aplicar
sheet_info = {
    'Country PM2.5 Exposure': {
        'filter_prefix': 'AVPMC_',
        'filter_cols': ['COUNTRY']
    },
    'Country PM2.5 Exceedance': {
        'filter_prefix': 'PMEXDC_',
        'filter_cols': ['COUNTRY']
    },
    'Urban PM2.5 Exposure': {
        'filter_prefix': 'AVPMU_',
        'filter_cols': ['COUNTRYENG']
    }
}

def load_and_filter_sheet(excel_data, sheet_name, filter_prefix, filter_cols):
    # Cargar la hoja
    df = excel_data.parse(sheet_name)
    # Filtrar las columnas que empiezan con el prefijo
    data_columns = [col for col in df.columns if col.startswith(filter_prefix)]
    # Renombrar las columnas eliminando el prefijo y espacios
    renamed_columns = {col: col.replace(filter_prefix, '').strip() for col in data_columns}
    df.rename(columns=renamed_columns, inplace=True)
    # Crear lista de columnas filtradas (sin prefijos)
    filtered_columns = filter_cols + list(renamed_columns.values())
    # Seleccionar las columnas filtradas
    filtered_df = df[filtered_columns]
    return filtered_df

# Diccionario para almacenar los DataFrames filtrados
filtered_dataframes = {}

# Cargar y filtrar cada hoja
for sheet_name, info in sheet_info.items():
    filtered_df = load_and_filter_sheet(
        excelData,
        sheet_name,
        info['filter_prefix'],
        info['filter_cols']
    )
    filtered_dataframes[sheet_name] = filtered_df

def get_filtered_data(sheet_name=None, country=None, year=None):
    """
    Devuelve los datos filtrados según los parámetros proporcionados.
    """
    results = []
    sheets_to_check = [sheet_name] if sheet_name else filtered_dataframes.keys()
    
    for sheet in sheets_to_check:
        df = filtered_dataframes.get(sheet)
        if df is not None:
            df_filtered = df.copy()
            
            # Filtrar por país
            if country is not None:
                country_col = None
                if 'COUNTRY' in df_filtered.columns:
                    country_col = 'COUNTRY'
                elif 'COUNTRYENG' in df_filtered.columns:
                    country_col = 'COUNTRYENG'
                else:
                    continue
                df_filtered = df_filtered[df_filtered[country_col] == country]
                if df_filtered.empty:
                    continue

            # Filtrar por año
            if year is not None:
                year = str(year)
                if year in df_filtered.columns:
                    columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + [year]
                    df_filtered = df_filtered[columns_to_keep]
                else:
                    continue
            else:
                # Incluir todas las columnas de años
                year_columns = [col for col in df_filtered.columns if col.isdigit()]
                columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + year_columns
                df_filtered = df_filtered[columns_to_keep]
            
            # Añadir el nombre de la hoja
            df_filtered['Sheet'] = sheet
            results.append(df_filtered)
    
    if results:
        df_result = pd.concat(results, ignore_index=True)
        return df_result.to_json(orient='records')
    else:
        return json.dumps([])


/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [65]:
# Ejemplos de uso:

# 1. Filtrar por año
json_por_ano = get_filtered_data(year=2015)
print("Datos para el año 2015:")
print(json_por_ano)

Datos para el año 2015:
[{"COUNTRY":"Afghanistan","2015":15.2331373925,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Albania","2015":11.8006311055,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Algeria","2015":8.6235388136,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"American Samoa","2015":0.2972666079,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Andorra","2015":5.4824161903,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Angola","2015":17.9665770518,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Anguilla","2015":0.438389564,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Antigua and Barbuda","2015":0.5475208233,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Argentina","2015":8.5046412815,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Armenia","2015":18.1509586835,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Aruba

In [66]:
# 2. Filtrar por país
json_por_pais = get_filtered_data(country='Colombia')
print("\nDatos para Colombia:")
print(json_por_pais)


Datos para Colombia:
[{"COUNTRY":"Colombia","2008":6.5176484937,"2009":7.4727359403,"2010":9.7057362387,"2011":6.3935288919,"2012":9.9906223802,"2013":8.2327534953,"2014":7.6382660527,"2015":7.1180866308,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null,"1998":null,"1999":null,"2000":null,"2001":null,"2002":null,"2003":null,"2004":null,"2005":null,"2006":null,"2007":null,"2016":null},{"COUNTRY":"Colombia","2008":2.4556198785,"2009":2.8382164135,"2010":8.6384392154,"2011":2.3437387644,"2012":8.8354609218,"2013":5.9500808999,"2014":2.9659934105,"2015":2.6041372727,"Sheet":"Country PM2.5 Exceedance","COUNTRYENG":null,"1998":null,"1999":null,"2000":null,"2001":null,"2002":null,"2003":null,"2004":null,"2005":null,"2006":null,"2007":null,"2016":null},{"COUNTRY":null,"2008":0.9434782681,"2009":1.1521739182,"2010":0.7521739136,"2011":0.8217391268,"2012":1.0913043515,"2013":0.9043478266,"2014":0.9434782681,"2015":0.939130441,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia","1998":1.6391

In [67]:
# 3. Filtrar por hoja
json_por_hoja = get_filtered_data(sheet_name='Country PM2.5 Exposure')
print("\nDatos de la hoja 'Country PM2.5 Exposure':")
print(json_por_hoja)


Datos de la hoja 'Country PM2.5 Exposure':
[{"COUNTRY":"Afghanistan","2008":13.2613703711,"2009":10.5070193678,"2010":11.9254917993,"2011":11.9669335239,"2012":13.4089859372,"2013":13.0235426258,"2014":12.2029002695,"2015":15.2331373925,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"Albania","2008":9.9145624185,"2009":9.0140325231,"2010":9.7203388625,"2011":9.9902603705,"2012":10.2651756651,"2013":9.6878014197,"2014":9.3546532804,"2015":11.8006311055,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"Algeria","2008":6.1386496386,"2009":4.6626322559,"2010":5.4414866863,"2011":5.2902802425,"2012":5.601431021,"2013":5.3973196964,"2014":6.053068689,"2015":8.6235388136,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"American Samoa","2008":0.0996477096,"2009":0.0995110127,"2010":0.0995110127,"2011":0.0995110127,"2012":0.0995110127,"2013":0.1012694136,"2014":0.1990220253,"2015":0.2972666079,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"Andorra","2008":5.6950730744,"2009":5.4800212025,"2010":5.173324

In [68]:

# 4. Filtrar por país y año
json_por_pais_ano = get_filtered_data(country='Colombia', year=2015)
print("\nDatos para Colombia en 2015:")
print(json_por_pais_ano)


Datos para Colombia en 2015:
[{"COUNTRY":"Colombia","2015":7.1180866308,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Colombia","2015":2.6041372727,"Sheet":"Country PM2.5 Exceedance","COUNTRYENG":null},{"COUNTRY":null,"2015":0.939130441,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":1.4339999843,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.1616822214,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.3420289755,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":3.2020334817,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":3.0508009116,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.7960244627,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":3.0,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.921000011,"Sheet":"Urban PM2

In [69]:
# 5. Filtrar por país, año y hoja
json_por_pais_ano_hoja = get_filtered_data(sheet_name='Country PM2.5 Exposure', country='Colombia', year=2015)
print("\nDatos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':")
print(json_por_pais_ano_hoja)


Datos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':
[{"COUNTRY":"Colombia","2015":7.1180866308,"Sheet":"Country PM2.5 Exposure"}]


In [70]:
# 6. Filtrar por país, año y hoja distinta
json_por_pais_ano_hoja = get_filtered_data(sheet_name='Urban PM2.5 Exposure', country='Colombia', year=2015)
print("\nDatos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':")
print(json_por_pais_ano_hoja)


Datos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':
[{"COUNTRYENG":"Colombia","2015":0.939130441,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":1.4339999843,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":2.1616822214,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":2.3420289755,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":3.2020334817,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":3.0508009116,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":2.7960244627,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":3.0,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":2.921000011,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":3.7952380937,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":3.2074073774,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colombia","2015":2.9086207069,"Sheet":"Urban PM2.5 Exposure"},{"COUNTRYENG":"Colom